# Loading and Storing the Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
raw_df = pd.read_csv("online_retail_II.csv")
df = raw_df

In [ ]:
df

# Initial Exploration

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()/1067371*100

# Data Cleaning Rule

**Cleaning Philosophy**
| Thing             | Action             |
| ----------------- | ------------------ |
| Returns           | Keep & flag        |
| Cancellations     | Keep & flag        |
| Invalid prices    | Drop Logically     |
| Missing customers | Keep & flag        |
| Non-products      | Keep & flag        |
| Time              | Normalize & expand |

Phase A: Structural Fixes (Do this first, no exceptions)

1️⃣ Create a working copy

Keep your raw dataframe untouched

Create a new dataframe for cleaning

Why:
You will mess something up. This saves you from self-hatred.

2️⃣ Standardize column names

Lowercase everything

Replace spaces with underscores

Remove weird characters

Example outcome:

Customer ID → customer_id

InvoiceDate → invoice_date

Why:
SQL and Python don’t like pretty names. They like boring names.

3️⃣ Fix data types (CRITICAL)

Do these in this exact order:

a) Invoice

Ensure it’s string

Do NOT try to make it numeric

b) StockCode

String

c) Description

String

Strip whitespace

Lowercase

d) InvoiceDate

Convert to datetime

Verify conversion worked (check nulls after conversion)

e) Customer ID

Convert from float to nullable integer OR string

Do NOT lose NaNs

Why:
This step makes the dataset honest about what it contains.

Phase B: Business Logic Flags (Very Important)

Now we add meaning without deleting data.

4️⃣ Identify cancellations

Create a boolean column:

is_cancelled_invoice

Logic: invoice starts with "C"

Why:
Cancellations are not errors. They’re events.

5️⃣ Identify returns

Create a boolean column:

is_return

Logic: quantity < 0

Why:
Returns matter for revenue and retention.

6️⃣ Identify guest customers

Create is_guest_customer

Logic: customer_id is null

Why:
Guest behavior ≠ returning customer behavior.

7️⃣ Identify non-product rows

Flag stock codes like:

POST

DOT

M

BANK CHARGES

Why:
These affect revenue but distort product analysis.

Phase C: Validity Rules (Where we finally drop stuff)

8️⃣ Handle invalid prices (carefully)

Apply this logic, not brute force:

If price <= 0 AND it is a sale → remove

If price <= 0 AND it is a return/cancellation → keep + flag

Why:
You already caught this. This preserves business reality.

9️⃣ Handle missing descriptions

Fill missing descriptions with a placeholder like:

"unknown_product"

Why:
Tiny number of rows. Not worth deleting.

Phase D: Derived Columns (Mandatory)

🔟 Create total price

total_price = quantity * price

Why:
Without this, revenue analysis is fake.

1️⃣1️⃣ Create time-based columns

From invoice_date, extract:

year

month

day

hour

Why:
SQL + Tableau will thank you later.

1️⃣2️⃣ Country grouping

Create country_group

Values:

"UK"

"Non-UK"

Why:
Business is UK-focused. This simplifies analysis.

Phase E: Post-Clean Sanity Check (DO NOT SKIP)

After all operations:

1️⃣3️⃣ Re-run:

shape

info

isna().sum()

describe()

Then ask yourself:

Did row count change? Why?

Did missing values increase? Why?

Does quantity + price still make sense?

Write short notes. These become interview gold.

# Implementing Data Cleaning

# Phase A : Structural Fixes

In [ ]:
df.rename(columns={'Invoice':'invoice', 'StockCode':'stockcode', 'Description':'description', 'Quantity':'quantity', 'InvoiceDate':'invoice_date', 'Price':'price', 'Customer ID':'customer_id', 'Country':'country'}, inplace=True)

df.columns
# here the columns had been renamed for sql, python and tableau.

In [ ]:
df['description'] = df['description'].astype(str).str.strip().str.lower()
# here all the discreption values had been converted to lower case and stripped them to avoid unnecessary leading and trailing spaces and lower case was choosed for standardization.

In [ ]:
df['invoice_date'] = pd.to_datetime(df['invoice_date'])
# here as invoice date was in object data type so it has to be converted into its respective data type

In [ ]:
df['customer_id'] = pd.to_numeric(df['customer_id']).astype(pd.Int64Dtype())
# As the customer id was in float like idk why but yeah, so it has been converted to int

# Phase B : Business Logic Flags

In [ ]:
df['is_cancelled_invoice'] = df['invoice'].astype(str).str.startswith('C')
# An boolean column was created in order to flag values if the invoice starts with C so it is a canceled order, which is stored as true in this column

In [ ]:
df['is_return'] = np.where(df['quantity']<0, True, False)
# An boolean column was created in order to flag values, if the Quantity is in negative so it is returned, which is stored as true in this column

In [ ]:
df['is_guest_customer'] = df['customer_id'].isna()
# An boolean column was created in order to flag values, if the customer id is not available, so this order was done by guest customer, which is stored as true in this column

In [ ]:
df['stockcode'] = df['stockcode'].astype(str).str.strip().str.upper()
non_product_codes = ['POST', 'DOT', 'M', 'BANK CHARGES']
df['is_non_product'] = df['stockcode'].isin(non_product_codes)
# An boolean column was created in order to flag values, if the stockcode is in 'POST', 'DOT', 'M', 'BANK CHARGES', so it is not a product, which is stored as true in this column

# Phase C : Validity Rules

In [ ]:
df['is_non_sales'] = df['is_cancelled_invoice'] | df['is_return']
# This is a boolean column which shows is the record a sale or not, As if the invoice starts with C or the quantity is in -tive so it is a non sales record
df['is_invalid_price'] = df['price'] <= 0
# This is again a boolean column where is the price is -tive it is an invalid price, either it is a returned order or data entry mistake

In [ ]:
df = df[~((df['is_non_sales'] == False) & (df['is_invalid_price'] == True))]
# It kept only those rows where price are either +tive or a returned order, It removed 2700 rows as they where error in data entry

In [ ]:
df['description'] = df['description'].replace('nan', 'unknown product')
# Replaced Nan which was converted into 'nan' to 'Unknown Product"

# Phase D : Derived Columns

In [ ]:
df['total_price'] = df['price'] * df['quantity']
# Total Price column was created in order to get the Revenue

In [ ]:
df['year'] = df['invoice_date'].dt.year
df['month'] = df['invoice_date'].dt.month
df['day'] = df['invoice_date'].dt.day
df['hour'] = df['invoice_date'].dt.hour
# Whole Date column has been chunked to get better features for time series analysis

In [ ]:
df['country_group'] = np.where(df['country']=='United Kingdom','UK','Non UK')
# Created a group of UK and Non UK country

# Phase E : Sanity Check

In [ ]:
df.shape
# Earlier there was 10,67,371 Rows and 8 columns, Now we have 10,64,621 Rows and 20 columns, As this data and our goal doesn't needed to remove values so we removed only 2,750 rows (invalid prices), but this data and our goal needed flags and labels so we created 12 new columns

In [ ]:
df.describe()
#  Here we have the same Stats but now the Min of Price 0
# quantity	invoice_date	price	customer_id	total_price	year	month	day	hour
# count	1.064621e+06	1064621	1.064621e+06	824293.0	1.064621e+06	1.064621e+06	1.064621e+06	1.064621e+06	1.064621e+06
# mean	9.727620e+00	2011-01-03 00:29:55.780188160	4.810442e+00	15324.734358	1.826559e+01	2.010426e+03	7.497212e+00	1.513415e+01	1.303312e+01
# min	-8.099500e+04	2009-12-01 07:45:00	0.000000e+00	12346.0	-1.684696e+05	2.009000e+03	1.000000e+00	1.000000e+00	6.000000e+00
# 25%	1.000000e+00	2010-07-09 11:44:00	1.250000e+00	13975.0	3.750000e+00	2.010000e+03	5.000000e+00	8.000000e+00	1.100000e+01
# 50%	3.000000e+00	2010-12-07 15:28:00	2.100000e+00	15257.0	9.900000e+00	2.010000e+03	8.000000e+00	1.500000e+01	1.300000e+01
# 75%	1.000000e+01	2011-07-22 12:20:00	4.150000e+00	16797.0	1.770000e+01	2.011000e+03	1.100000e+01	2.200000e+01	1.500000e+01
# max	8.099500e+04	2011-12-09 12:50:00	3.897000e+04	18287.0	1.684696e+05	2.011000e+03	1.200000e+01	3.100000e+01	2.100000e+01
# std	1.706573e+02	NaN	9.555657e+01	1697.434289	2.820495e+02	5.737234e-01	3.526255e+00	8.679866e+00	2.435686e+00


In [ ]:
df.info()
# Here we have correct data types for each column

In [ ]:
df.isna().sum()
# Here we have No nulls except customer_id which have 2,40,328 nulls which are guest customers

In [ ]:
cleaned_df = df
cleaned_df.to_csv("cleaned_online_retail.csv",index=False)